# Discogs Functions

In [111]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogs import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-07-16 22:53:35.719785


# Discogs Base Class

In [112]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Warning! Local Discog Directory /Users/tgadfort/Documents/music/Discog is not Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/special exists
/Volumes/Music/Discog/artist-special exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value


# Collection Section

In [85]:
%load_ext autoreload
%autoreload
col = collections(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
#col = collections(disc)
decades=['1920', '1930', '1940', '1950', '1960', '1970', '1980', '1990', '2000', '2010']
decades=['1960', '1970', '1980', '1990', '2000', '2010']
fmat="Compilation"
country="US"
style="Soundtrack"
for decade in decades:
    col.downloadCollection(maxPages=30, style=style, country="US", decade=decade) #, fmat=fmat)
#for year in range(1950, 2018):
#    col.downloadCollectionsByYear(maxPages=2, year=str(year), Ncountries=50, Ngenres=100, Nstyles=100)

#https://www.discogs.com/search/?style_exact=Soundtrack&format_exact=Compilation&limit=250&page=1&country_exact=US
#https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&layout=sm&style_exact=Soundtrack&country_exact=US&format_exact=Compilation&decade=2010&page=1
#https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&layout=sm&style_exact=Soundtrack&country_exact=US&format_exact=Compilation&decade=2010&page=1

SaveVal -> US-1960-Soundtrack
SaveVal -> US-1970-Soundtrack
SaveVal -> US-1980-Soundtrack
SaveVal -> US-1990-Soundtrack
SaveVal -> US-2000-Soundtrack
SaveVal -> US-2010-Soundtrack


In [ ]:
ifile = "/Volumes/Music/Discog/collections/US-1960-Soundtrack-9.p"
from webUtils import getWebData, getHTML
verydebug=False
fdata  = getFile(ifile)
bsdata = getHTML(fdata)

albumsDir = disc.getAlbumsDir()

albs = albums(disc)


if True:
    collectionsDir  = col.getCollectionsDir()
    collectionFiles = findExt(collectionsDir, ext=".p")
    print("Found {0} downloaded collection files".format(len(collectionFiles)))
    for i,ifile in enumerate(collectionFiles):
        print("\n\n\n")
        print("Parsing {0}/{1}: {2}".format(i,len(collectionFiles), ifile))
        
        bsdata = getHTML(ifile)
        artistDB  = {}

        h4s = bsdata.findAll("h4")
        if verydebug:
            print("Found {0} h4 tags".format(len(h4s)))
        
        for ih4,h4 in enumerate(h4s):
            spans = h4.findAll("span")
            
            #### Get Artist
            ref   = None
            if len(spans) == 0:
                ref = h4.find("a")
            else:
                ref = spans[0].find("a")
                
            if ref is None:
                continue
                
            try:
                artisthref = ref.attrs.get('href')
                artist     = ref.text.strip()
            except:
                print("Could not get artist/href from {0}".format(ref))
                continue
                
            
            #### Get Album
        
            try:
                ref        = h4.find("a", {"class": "search_result_title"})
                albumhref  = ref.attrs.get('href')
                album      = ref.text.strip()
            except:
                print("Could not get album/href from {0}".format(ref))
                continue

            
            #print("Artist: {0} ---> {1}".format(artist,artisthref))
            #print("Album:  {0} ---> {1}".format(album,albumhref))

            artistID  = arts.discogsUtils.getArtistID(artisthref)
            modValue  = arts.discogsUtils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())
            
            
            ##### Download Artist Data
            url      = arts.getArtistRef(artisthref)
            savename = arts.getArtistSavename(artistID)
            if savename is not None:
                if not isFile(savename):
                    try:
                        arts.downloadArtistURL(url, savename)
                    except:
                        continue
                #print('\t',url)
                #print('\t',savename)
                
        
            if modValue is None:
                artistModDir = setDir(albumsDir, "NAN")
                artistIDDir = setDir(artistModDir, "NAN")
            else:
                artistModDir = setDir(albumsDir, str(modValue))
                artistIDDir = setDir(artistModDir, artistID)


            ##### Download Album Data
            try:
                mediaID = int(getBasename(albumhref))
            except:
                raise ValueError("Not sure about {0}".format(h4))

            #print('\tAlbumDir:',artistIDDir)
            #print('\tMediaID: ',mediaID)

            if not isDir(artistIDDir):
                mkDir(artistIDDir)
            savename = setFile(artistIDDir, "{0}.p".format(mediaID))
            #print('\tSavename:',savename)
            if isFile(savename):
                continue
     
            #print('\t',savename)
            #print("")
            #continue

            baseURL = disc.discogURL
            url     = urllib.parse.urljoin(baseURL, quote(albumhref))
            try:
                albs.downloadAlbumURL(url, savename)
            except:
                continue

Found 3280 downloaded collection files




Parsing 0/3280: /Volumes/Music/Discog/collections/Argentina-1950-Blues-1.p
Could not get album/href from None




Parsing 1/3280: /Volumes/Music/Discog/collections/Argentina-1950-Blues-2.p
Could not get album/href from None




Parsing 2/3280: /Volumes/Music/Discog/collections/Argentina-1950-Brass & Military-1.p
Could not get album/href from None




Parsing 3/3280: /Volumes/Music/Discog/collections/Argentina-1950-Brass & Military-2.p
Could not get album/href from None




Parsing 4/3280: /Volumes/Music/Discog/collections/Argentina-1950-Children's-1.p
Could not get album/href from None




Parsing 5/3280: /Volumes/Music/Discog/collections/Argentina-1950-Children's-2.p
Could not get album/href from None




Parsing 6/3280: /Volumes/Music/Discog/collections/Argentina-1950-Classical-1.p
Could not get album/href from None




Parsing 7/3280: /Volumes/Music/Discog/collections/Argentina-1950-Classical-2.p
Could not get album/href from None




Parsi

Saving /Volumes/Music/Discog/albums/42/4096442/8034335.p
  --> This file is 16.0kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Graeme-Bells-Australian-Jazz-Band-Wolverine-Blues-Sobbin-Blues/master/1238660
Saving /Volumes/Music/Discog/albums/9/3701309/1238660.p
  --> This file is 14.5kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Vladimir-Karsan-The-Harry-Lime-Theme-The-Cafe-Mozart-Waltz/release/13222047
Saving /Volumes/Music/Discog/albums/58/6989058/13222047.p
  --> This file is 14.8kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Southern-Jazz-Group-Clarinet-Spice-Stomp-Miss-Hannah/release/10250665
Saving /Volumes/Music/Discog/albums/92/4827192/10250665.p
  --> This file is 15.6kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Ethel-Smith-And-The-Bando-Carioca-Tico-Tico/release/9193249
Saving /Volumes/Music/Discog/albums/72/527972/9193249.p
  --> This file is 17.1kB.
Done. Sleeping for 2.5 s

Downloading: https://www.discogs.com/Frank-Sinatra-My-Cousin-Louella-Everybody-Loves-Somebody/release/11946630
Saving /Volumes/Music/Discog/albums/33/52833/11946630.p
  --> This file is 14.8kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Louis-Armstrong-With-Gordon-Jenkins-And-His-Orchestra-Ella-Fitzgerald-And-Sy-Oliver-Blueberry-Hill-B/release/11885933
Saving /Volumes/Music/Discog/albums/1/38201/11885933.p
  --> This file is 16.4kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Graeme-Bell-His-Australian-Jazz-Band-I-Wish-I-Could-Shimmy-Like-My-Sister-Kate-Fidgety-Feet/release/10040791
Saving /Volumes/Music/Discog/albums/9/3701309/10040791.p
  --> This file is 15.3kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Frank-Sinatra-The-Huckle-Buck-Dont-Cry-Joe/release/11946661
Saving /Volumes/Music/Discog/albums/33/52833/11946661.p
  --> This file is 14.9kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.co

Saving /Volumes/Music/Discog/albums/18/258518/11885866.p
  --> This file is 16.1kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Spike-Jones-And-His-City-Slickers-McNamaras-Band-Knock-Knock/release/8986087
Saving /Volumes/Music/Discog/albums/68/287068/8986087.p
  --> This file is 14.9kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Joan-Wilton-Lawrie-Brooks-The-Lester-Sisters-Beautiful-Brown-Eyes-My-Kind-Of-Country/release/6941063
Saving /Volumes/Music/Discog/albums/27/4375627/6941063.p
  --> This file is 15.5kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Elsie-Findlay-Haere-Ra-Long-Long-Ago/release/13427861
Saving /Volumes/Music/Discog/albums/88/7073888/13427861.p
  --> This file is 14.9kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Dawn-Lake-With-The-Gus-Merzi-Quintette-They-Cant-Make-A-Lady-Out-Of-Me-Mountain-Gal/release/10013892
Saving /Volumes/Music/Discog/albums/37/3991537/10013892.p
 

Downloading: https://www.discogs.com/Horst-Winter-La-Vie-En-Rose-Negermamas-Wiegenlied/master/1544850
Saving /Volumes/Music/Discog/albums/63/573463/1544850.p
  --> This file is 14.9kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Horst-Winter-La-Vie-En-Rose-Negermamas-Wiegenlied/release/12232992
Saving /Volumes/Music/Discog/albums/63/573463/12232992.p
  --> This file is 16.6kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Orchester-Kurt-Edelhagen-Caravan-Percussion-In-Bop/release/3135490
Saving /Volumes/Music/Discog/albums/99/312599/3135490.p
  --> This file is 14.7kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/F-Charles-Adler-Wiener-Staatsopernchor-Die-Dreigroschenoper/release/11693283
Saving /Volumes/Music/Discog/albums/38/996438/11693283.p
  --> This file is 14.8kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Orchester-Kurt-Edelhagen-Infection-Theme-On-Drums/release/13339856
Saving /Volume

...

Saving /Volumes/Music/Discog/albums/31/3896031/10616649.p
  --> This file is 18.3kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/artist/5154905-Silent-Partner-5
Saving /Volumes/Music/Discog/artists/5/5154905.p
  --> This file is 15.1kB.
Done. Sleeping for 2 seconds
Downloading: https://www.discogs.com/Silent-Partner-Collection-I/release/10347093
Saving /Volumes/Music/Discog/albums/5/5154905/10347093.p
  --> This file is 16.8kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Various-KudatahVol-2/release/10060131
Saving /Volumes/Music/Discog/albums/NAN/NAN/10060131.p
  --> This file is 18.2kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Sangam-Disconsolate/master/1271559
Saving /Volumes/Music/Discog/albums/13/2740413/1271559.p
  --> This file is 16.7kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/LIGHTS-Skin-Earth/release/10902176
Saving /Volumes/Music/Discog/albums/62/1397662/10902176.p
  --> Th

Downloading: https://www.discogs.com/Jamiroquai-Automaton/release/10067895
Saving /Volumes/Music/Discog/albums/29/8029/10067895.p
  --> This file is 19.3kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Sangam-Disconsolate/release/11176137
Saving /Volumes/Music/Discog/albums/13/2740413/11176137.p
  --> This file is 17.8kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Various-Neo-Classix/master/1408465
Saving /Volumes/Music/Discog/albums/NAN/NAN/1408465.p
  --> This file is 18.7kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Waterfront-Dining-Mavi/master/1500824
Saving /Volumes/Music/Discog/albums/26/4407026/1500824.p
  --> This file is 15.1kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/David-Pritchard-Nocturnal-Earthworm-Stew-Bouillabaisse-Nocturne-Aux-Vers-De-Terre/release/10234484
Saving /Volumes/Music/Discog/albums/65/1224465/10234484.p
  --> This file is 19.3kB.
Done. Sleeping for 2.5 seco

Downloading: https://www.discogs.com/TWRP-Ladyworld/release/10451571
Saving /Volumes/Music/Discog/albums/42/2479942/10451571.p
  --> This file is 18.7kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/artist/3292990-Argus-6
Saving /Volumes/Music/Discog/artists/90/3292990.p
  --> This file is 24.1kB.
Done. Sleeping for 2 seconds
Downloading: https://www.discogs.com/Argus-The-Architect-Of-Time/master/1155523
Saving /Volumes/Music/Discog/albums/90/3292990/1155523.p
  --> This file is 16.5kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/artist/3267745-Dimitri-Veimar
Saving /Volumes/Music/Discog/artists/45/3267745.p
  --> This file is 19.4kB.
Done. Sleeping for 2 seconds
Downloading: https://www.discogs.com/Dimitri-Veimar-Agung/master/1368574
Saving /Volumes/Music/Discog/albums/45/3267745/1368574.p
  --> This file is 15.3kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Korea-Undok-Group-Deepening-Floor/release/10595617
Savin

Downloading: https://www.discogs.com/Nitin-Sunday-Grey-EP/release/10243222
Saving /Volumes/Music/Discog/albums/49/1370849/10243222.p
  --> This file is 15.4kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Earthen-Sea-A-Relentless-Gaze/release/10346699
Saving /Volumes/Music/Discog/albums/9/677309/10346699.p
  --> This file is 17.4kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/artist/5016704-TiMT
Saving /Volumes/Music/Discog/artists/4/5016704.p
  --> This file is 17.8kB.
Done. Sleeping for 2 seconds
Downloading: https://www.discogs.com/TiMT-Time-Flies-Like-An-Arrow/release/9984597
Saving /Volumes/Music/Discog/albums/4/5016704/9984597.p
  --> This file is 17.0kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Various-Coral-City-Vol-2/release/11451832
Saving /Volumes/Music/Discog/albums/NAN/NAN/11451832.p
  --> This file is 16.6kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Gammer-The-Drop/master/

Downloading: https://www.discogs.com/artist/2256632-Peter-Peter-3
Saving /Volumes/Music/Discog/artists/32/2256632.p
  --> This file is 20.1kB.
Done. Sleeping for 2 seconds
Downloading: https://www.discogs.com/Peter-Peter-Noir-%25C3%2589den/release/10525017
Saving /Volumes/Music/Discog/albums/32/2256632/10525017.p
  --> This file is 17.3kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/artist/4468570-unfollow
Saving /Volumes/Music/Discog/artists/70/4468570.p
  --> This file is 16.7kB.
Done. Sleeping for 2 seconds
Downloading: https://www.discogs.com/unfollow-Blader-Hater/release/10936120
Saving /Volumes/Music/Discog/albums/70/4468570/10936120.p
  --> This file is 15.5kB.
Done. Sleeping for 2.5 seconds
Downloading: https://www.discogs.com/Maylee-Todd-Acts-Of-Love/master/1298226
Saving /Volumes/Music/Discog/albums/18/507518/1298226.p
  --> This file is 16.6kB.
Done. Sleeping for 2.5 seconds




Parsing 185/3280: /Volumes/Music/Discog/collections/Canada-2017-Folk, Wor

In [83]:
#col.createCollectionsData()
#col.downloadCollections()
#col.parseCollections()
#col.mergeCollections(debug=True, force=True)
#col.createCollectionDBs(debug=True)

In [27]:
disc.moveAlbumFilesToNewModValue(newModValue=100, oldModValue=500)

Moving 560 directories from 100 to 0
/Volumes/Music/Discog/albums/100/1002600
/Volumes/Music/Discog/albums/100


ZeroDivisionError: division by zero

# Artist Section

In [ ]:
db = disc.getArtistNameToIDsData()

# Albums Metadata

In [ ]:
artistNameToID   = disc.getArtistNameToIDData()
artistNameToIDs  = disc.getArtistNameToIDsData()
artistIDToName   = disc.getArtistIDToNameData()        
artistRefToID    = disc.getArtistRefToIDData()
artistIDToRef    = disc.getArtistIDToRefData()
artistRefToName  = disc.getArtistRefToNameData()
artistNameToRef  = disc.getArtistNameToRefData()
artistNameToRefs = disc.getArtistNameToRefsData()

In [ ]:
albumRefToName  = disc.getAlbumRefToNameData()
albumNameToRef  = disc.getAlbumNameToRefData()

artistIDCoreAlbumIDs   = disc.getArtistIDCoreAlbumNames()
artistIDAlbumIDs       = disc.getArtistIDAlbumNames()



In [ ]:
from collections import Counter
core = ["Albums"]
albumCntr = Counter()


print("{0: <4}{1: <50}{2: <15}{3: <15}{4: <12}{5: <12}{6: <12}{7: <12}{8: <12}{9: <12}".format(
    "#","File","Core IDs-IDs","IDs-IDs","Name-IDs","Name-Refs","ID-Names","ID-Refs","Ref-IDs","Ref-Names"))


for i,dbfile in enumerate(dbfiles):
    db = getFile(dbfile)
    for artistID,artistData in db.items():
        artist      = artistData['Artist']
        artistName  = dutils.getArtistName(artist)
        artistRef   = artistData['URL']
        #print("{0: <20}{1: <50}{2}".format(artistID,artist,artistRef))
        
    print("{0: <4}{1: <50}{2: <15}{3: <15}{4: <12}{5: <12}{6: <12}{7: <12}{8: <12}{9: <12}".format(
        i,dbfile,
        len(artistIDCoreAlbumIDs),len(artistIDAlbumIDs),
        len(albumNameToID),len(albumNameToRef),len(albumIDToName),
        len(albumIDToRef),len(albumRefToID),len(albumRefToName)))



In [ ]:
artistIDAlbumIDs

In [ ]:
artistIDAlbumsIDs

In [ ]:
artistNames = {}
artistYears = {}
artistDBDir = disc.getArtistsDBDir()   
files       = findExt(artistDBDir, ext='.p')
print(len(files))

In [ ]:
db["The Beatles"]

In [ ]:
#arts = artists(disc)
#art.findKnownArtists(debug=True)
#art.findUnknownArtists(minVal=200, debug=True)
#art.downloadUnknownArtists(debug=True)
#arts.parseArtistFiles()
#art.searchDiscogForArtist("$uicideboy$")
#arts.buildMetadata()

In [ ]:
class artistName:
    def __init__(self, bsdata):
        self.artist = None
        
        ## 1st Try
        result = bsdata.find("h1", {'class':'hide_desktop'})
        if result:
            artist = result.text
            if len(artist) > 0:
                self.artist = fixName(artist)

        ## 2nd Try
        result = bsdata.find("h1", {'class':'hide_mobile'})
        if result:
            artist = result.text
            if len(artist) > 0:
                self.artist = fixName(artist)

    def getArtist(self):
        return self.artist

In [ ]:
ifile="/Volumes/Music/Discog/artists/0/9900.p"
fdata = getHTML(getFile(ifile, version=2))
an = artistName(fdata)
an.getArtist()
#name = artistName(fdata)
#name


In [ ]:
bsdata = getHTML(ifile, version=2)
print(type(bsdata))
name = artistName(bsdata)

art = artists()
art.getArtistName(bsdata, True)
if False:
    retval["Artist"]      = self.getArtistName(bsdata, debug)
    retval["URL"]         = self.getArtistURL(bsdata, debug)
    retval["ID"]          = self.getArtistDiscID(retval["URL"], debug)
    retval["Pages"]       = self.getArtistPages(bsdata, debug)
    retval["Variations"]  = self.getArtistVariations(bsdata, debug)
    retval["MediaCounts"] = self.getArtistMediaCounts(bsdata, debug)
    retval["Media"]       = self.getArtistMedia(bsdata, debug)


In [ ]:
from ioUtils import getFile
from fsUtils import isFile, isDir
from webUtils import getHTML, isBS4
from strUtils import fixName

from discogs import discogs

    

class artist(discogs):
    def __init__(self):
        self.name = "artist"
        
        
    ######################################################################
    ## Get Data From Artist File From Discogs
    ######################################################################
    def getData(self, inputdata):
        if isinstance(inputdata, str):
            if isFile(inputdata):
                bsdata = getHTML(getFile(fname))
            else:
                raise ValueError("Not sure about string input: {0}".format(inputdata))
        elif isBS4(inputdata):
            bsdata = inputdata
            pass
        else:
            raise ValueError("Not sure about input type: {0}".format(type(inputdata)))

        self.bsdata = bsdata
        
        return self.parse()


    def parse(self, debug = False):
        bsdata = self.bsdata
        
        retval = {}
        retval["Artist"]      = self.getArtistName(bsdata, debug)
        retval["URL"]         = self.getArtistURL(bsdata, debug)
        retval["ID"]          = self.getArtistDiscID(retval["URL"], debug)
        retval["Pages"]       = self.getArtistPages(bsdata, debug)
        retval["Variations"]  = self.getArtistVariations(bsdata, debug)
        retval["MediaCounts"] = self.getArtistMediaCounts(bsdata, debug)
        retval["Media"]       = self.getArtistMedia(bsdata, debug)

        #print retval
        return retval    
        
        
    def getNamesAndURLs(self, content):
        data = []
        for ref in content.findAll("a"):
            url    = ref.attrs['href']
            name   = ref.text

            discID = self.getArtistDiscID(url)
            data.append([name,url,discID])                    
        return data



    def getArtistDiscID(self, suburl, debug = False):
        ival = "/artist"
        if not isinstance(suburl, str) and not isinstance(suburl, unicode):
            return None

        pos = suburl.find(ival)
        if pos == -1:
            return None

        data = suburl[pos+len(ival)+1:]
        pos  = data.find("-")
        discID = data[:pos]
        try:
            int(discID)
        except:
            return None

        return str(discID)



    def getArtistMediaCounts(self, bsdata, debug = False):
        mediaCounts = {}
        results = bsdata.findAll("ul", {"class": "facets_nav"})
        for result in results:
            for li in result.findAll("li"):
                ref = li.find("a")
                if ref:
                    attrs = ref.attrs
                    span = ref.find("span", {"class": "facet_count"})
                    count = None
                    if span:
                        count = span.text
                        credittype    = attrs.get("data-credit-type")
                        creditsubtype = attrs.get("data-credit-subtype")
                        if credittype and creditsubtype:
                            if mediaCounts.get(credittype) == None:
                                mediaCounts[credittype] = {}
                            if mediaCounts[credittype].get(creditsubtype) == None:
                                mediaCounts[credittype][creditsubtype] = count

        return mediaCounts





    def getArtistURL(self, bsdata, debug = False):
        # 1st Try
        result = bsdata.find("link", {"rel": "canonical"})
        if result:
            url = result.attrs["href"]
            url = url.replace("https://www.discogs.com", "")
            if url.find("/artist/") > -1:
                return url

        # 2nd Try
        result = bsdata.find("link", {"hreflang": "en"})
        if result:
            url = result.attrs["href"]
            url = url.replace("https://www.discogs.com", "")
            if url.find("/artist/") > -1:
                return url            

        return None    


    def getArtistMediaAlbum(self, td, debug = False):
        retval = {"URL": None, "Album": None, "Format": None}
        for span in td.findAll("span"):
            attrs = span.attrs
            if attrs.get("class"):
                if 'format' in attrs["class"]:
                    albumformat = span.text
                    albumformat = albumformat.replace("(", "")
                    albumformat = albumformat.replace(")", "")
                    retval["Format"] = albumformat
                    continue
            span.replaceWith("")

        ref = td.find("a")
        if ref:
            retval["URL"]   = ref.attrs['href']
            retval["Album"] = ref.text

        return retval


    def getArtistMedia(self, bsdata, debug = False):
        table = bsdata.find("table", {"id": "artist"})
        if table == None:
            return None

        media = {}
        name  = None
        for tr in table.findAll("tr"):
            h3 = tr.find("h3")
            if h3:
                name = h3.text
                media[name] = []
                continue


            # Album, Class, Format
            result = tr.find("td", {"class": "title"})
            album  = None
            url    = None
            albumformat = name
            if result:
                retval      = self.getArtistMediaAlbum(result)
                album       = fixName(retval.get("Album"))
                url         = retval.get("URL")
                albumformat = retval.get("Format")

            if album == None:
                continue

            # Code
            code = tr.attrs.get("data-object-id")

            # AlbumClass
            albumclass = tr.attrs.get("data-object-type")

            # AlbumURL
            result  = tr.find("td", {"class": "artist"})
            artists = None
            if result:
                artists = self.getNamesAndURLs(result)

            # Year
            result = tr.find("td", {"class": "year"})
            year   = None
            if result:
                year = result.text

            data = {}
            data["Album"]  = album
            data["URL"]    = url
            data["Class"]  = albumclass
            data["Format"] = albumformat
            data["Artist"] = artists
            data["Code"]   = code
            data["Year"]   = year
            media[name].append(data)
            #if debug: print "  Found album:",album,"of type:",name


        newMedia = {}
        for name,v in media.items():
            newMedia[name] = {}
            for item in v:
                code = item['Code']
                del item['Code']
                newMedia[name][code] = item

        media = newMedia

        return media


    def getArtistVariations(self, bsdata, debug = False):
        result = bsdata.find("div", {"class": "profile"})
        variations = {}
        if result:
            heads = result.findAll("div", {"class": "head"})
            heads = [x.text for x in heads]
            heads = [x.replace(":","") for x in heads]

            content = result.findAll("div", {"class": "content"})
            if len(heads) != len(content):
                raise("Mismatch in head/content")

            for i in range(len(heads)):
                if heads[i] == "Sites":
                    content[i] = self.getNamesAndURLs(content[i])
                elif heads[i] == "In Groups":
                    content[i] = self.getNamesAndURLs(content[i])
                elif heads[i] == "Variations":
                    content[i] = self.getNamesAndURLs(content[i])
                elif heads[i] == "Aliases":
                    content[i] = self.getNamesAndURLs(content[i])
                else:
                    content[i] = content[i].text
                    content[i] = content[i].strip()
                variations[heads[i]] = content[i]

        return variations



    def getArtistPages(self, bsdata, debug = False):
        result = bsdata.find("div", {"class": "pagination bottom "})
        total = 0
        num   = 0
        if result:
            pages = result.find("strong").text
            pages = pages.strip()
            pages = pages.split()[-1]
            pages = pages.replace(",", "")
            try:
                total = int(pages)
                num = int(ceil(float(total)/500))
            except:
                raise("Can not parse pages",pages)


        return num,total

# Albums

In [116]:
from fsUtils import setFile, isFile, setDir, isDir, mkDir, mkSubDir, setSubDir
from fileUtils import getBasename, getBaseFilename
from ioUtils import getFile, saveFile
from webUtils import getWebData, getHTML, getURL
from searchUtils import findExt, findPattern
from timeUtils import clock, elapsed, update
from collections import Counter
from math import ceil
from time import sleep

class albums():
    def __init__(self, discog, basedir=None):
        self.disc = discog
        self.name = "albums"
        
        ## General Imports
        self.getCodeDir          = self.disc.getCodeDir
        self.getAlbumsDir        = self.disc.getAlbumsDir
        self.getArtistsDBDir     = self.disc.getArtistsDBDir
        self.getDiscogDBDir      = self.disc.getDiscogDBDir
        self.discogsUtils        = discogsUtils()
        
        self.prevSearches        = {}
        
        self.modVal = self.disc.getMaxModVal
        
        self.starterDir = setDir(self.getCodeDir(), self.name)
        if not isDir(self.starterDir):
            print("Creating {0}".format(self.starterDir))
            mkDir(self.starterDir, debug=True)
        
        
    ######################################################################
    ### Load Full Artist Data 
    ######################################################################
    def downloadAlbumURL(self, url, savename, sleeptime=2.5):
        if isFile(savename):
            return
        
        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        headers={'User-Agent':user_agent,} 

        sleep(1)

        print("Downloading: {0}".format(url))

        request=urllib.request.Request(url,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read() # The data u need

        print("Saving {0}".format(savename))
        saveJoblib(data=data, filename=savename, compress=True)
        print("Done. Sleeping for {0} seconds".format(sleeptime))
        sleep(sleeptime)
        
        
    def downloadAlbumDataData(self, modval, mediaTypes=["Albums"], maxAlbums=None, debug=False):
        maxModVal = self.modVal
        
        albumsDir = self.getAlbumsDir()
        
        dbname = setFile(self.getArtistsDBDir(), "{0}-DB.p".format(modval))
        dbdata = getFile(dbname, version=3)
        
        artistModDir = setSubDir(albumsDir, str(modval))
            
        nArtists = len(dbdata)
        iArtists = 0
        for artistID, artistData in dbdata.items():
            iArtists += 1
            if artistID in ['712500', '1745000']:
                continue
            #retval = {}
            #retval["Artist"]      = self.getArtistName(bsdata, debug)
            #retval["URL"]         = self.getArtistURL(bsdata, debug)
            #retval["ID"]          = self.getArtistDiscID(retval["URL"], debug)
            #retval["Pages"]       = self.getArtistPages(bsdata, debug)
            ##retval["Variations"]  = self.getArtistVariations(bsdata, debug)
            #retval["MediaCounts"] = self.getArtistMediaCounts(bsdata, debug)
            #retval["Media"]       = self.getArtistMedia(bsdata, debug)
            
            print(iArtists,'/',nArtists,'\t:',artistData["ID"],'\t',artistData["Artist"])
            artistIDDir = setDir(artistModDir, artistID)
            media = artistData["Media"]
            for mediaType, mediaTypeData in media.items():
                if mediaTypes is not None:
                    if mediaType not in mediaTypes:
                        continue
                nget = 0
                for mediaID, mediaIDData in mediaTypeData.items():
                    albumName = mediaIDData["Album"]
                    albumArtist = mediaIDData["Artist"][0][0]
                    print("\t",mediaID,'\t',albumArtist,'\t',albumName)
                    savename = setFile(artistIDDir, "{0}.p".format(mediaID))
                    if isFile(savename):
                        continue
                    if maxAlbums is not None:
                        if nget >= maxAlbums:
                            break
     
                    baseURL = self.disc.discogURL
                    url = urllib.parse.urljoin(baseURL, quote(mediaIDData["URL"]))                    
                    self.downloadAlbumURL(url, savename)

In [8]:
import random
albs = albums(disc)
nErrs = False
while nErrs is False:
    nErrs = True
    albs.downloadAlbumDataData(int(random.uniform(0,disc.getMaxModVal()))) #, mediaTypes=None)        
    if False:
        nErrs = False
        print("Sleeping for 60 seconds...")
        sleep(60)
#art.findKnownArtists(debug=True)
#art.findUnknownArtists(minVal=200, debug=True)
#art.downloadUnknownArtists(debug=True)
#arts.parseArtistFiles()
#art.searchDiscogForArtist("$uicideboy$")
#arts.buildMetadata()

1 / 3021 	: 390652 	 Johnny Baylor
2 / 3021 	: 1754652 	 Geoff Hing
3 / 3021 	: 3104152 	 Shantala
	 7715212 	 Shantala 	 The Love Window


TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType